In [1]:
 ! pip install -q kaggle

In [2]:
 from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"suriyashanmugam","key":"fe9729dd1f43599cb51829e812329b52"}'}

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [6]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets list

ref                                                              title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
lainguyn123/student-performance-factors                          Student Performance Factors                          94KB  2024-09-02 10:53:57           8411        180  1.0              
prokshitha/home-value-insights                                   House Price Regression Dataset                       26KB  2024-09-06 15:15:12            970         29  1.0              
hanaksoy/customer-purchasing-behaviors                           Customer Purchasing Behaviors                         1KB  2024-09-01 22:18:07           4456         64  1.0              
hanaksoy/health-and-sleep-statistics                   

In [9]:
! kaggle competitions download -c 'california-house-prices'

 17% 5.00M/29.5M [00:00<00:00, 48.4MB/s]
100% 29.5M/29.5M [00:00<00:00, 163MB/s] 


In [10]:
! mkdir california-comp-files

In [11]:
! unzip california-house-prices.zip -d california-comp-files

Archive:  california-house-prices.zip
  inflating: california-comp-files/sample_submission.csv  
  inflating: california-comp-files/test.csv  
  inflating: california-comp-files/train.csv  


In [12]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 977.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requireme

In [2]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

directory = '/content/california-comp-files'  # directory where you have downloaded the data CSV files from the competition
label = 'Sold Price'  # name of target variable to predict in this competition
eval_metric = 'rmse'  # Optional: specify that competition evaluation metric is AUC
save_path = directory + 'AutoGluonModels/'  # where to store trained models

train_df = pd.read_csv(directory+'/train.csv')

In [3]:
train_df.head()

,Id,Address,Sold Price,Summary,Type,Year built,Heating,Cooling,Parking,Lot,...,Parking features,Tax assessed value,Annual tax amount,Listed On,Listed Price,Last Sold On,Last Sold Price,City,Zip,State
0,0,540 Pine Ln,3825000.0,"540 Pine Ln, Los Altos, CA 94022 is a single f...",SingleFamily,1969.0,"Heating - 2+ Zones, Central Forced Air - Gas","Multi-Zone, Central AC, Whole House / Attic Fan","Garage, Garage - Attached, Covered",1.0,...,"Garage, Garage - Attached, Covered",886486.0,12580.0,2019-10-24,4198000.0,NaN,NaN,Los Altos,94022,CA
1,1,1727 W 67th St,505000.0,"HURRY, HURRY.......Great house 3 bed and 2 bat...",SingleFamily,1926.0,Combination,"Wall/Window Unit(s), Evaporative Cooling, See ...","Detached Carport, Garage",4047.0,...,"Detached Carport, Garage",505000.0,6253.0,2019-10-16,525000.0,2019-08-30,328000.0,Los Angeles,90047,CA
2,2,28093 Pine Ave,140000.0,'THE PERFECT CABIN TO FLIP! Strawberry deligh...,SingleFamily,1958.0,Forced air,NaN,0 spaces,9147.0,...,NaN,49627.0,468.0,2019-08-25,180000.0,NaN,NaN,Strawberry,95375,CA
3,3,10750 Braddock Dr,1775000.0,Rare 2-story Gated 5 bedroom Modern Mediterran...,SingleFamily,1947.0,Central,Central Air,"Detached Carport, Driveway, Garage - Two Door",NaN,...,"Detached Carport, Driveway, Garage - Two Door",1775000.0,20787.0,2019-10-24,1895000.0,2016-08-30,1500000.0,Culver City,90230,CA
4,4,7415 O Donovan Rd,1175000.0,Beautiful 200 acre ranch land with several pas...,VacantLand,NaN,NaN,NaN,0 spaces,NaN,...,NaN,NaN,NaN,2019-06-07,1595000.0,2016-06-27,900000.0,Creston,93432,CA


In [5]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_df, presets='medium_quality', time_limit=300
)

results = predictor.fit_summary()

Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
GPU Count:          0
Memory Avail:       11.10 GB / 12.67 GB (87.6%)
Disk Space Avail:   65.26 GB / 107.72 GB (60.6%)
Presets specified: ['medium_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': False}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': False,
 'calibrate': 'auto',
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_ray_logging': True,
             'holdout_data': None,
             'holdout_frac': 0.1111111111111111,
             'memory_safe_fits': True,
             'n_folds': 2,
             'n_r

[50]	valid_set's rmse: 487452
[100]	valid_set's rmse: 421358
[150]	valid_set's rmse: 401205
[200]	valid_set's rmse: 390638
[250]	valid_set's rmse: 383412
[300]	valid_set's rmse: 376279
[350]	valid_set's rmse: 376473
[400]	valid_set's rmse: 375263
[450]	valid_set's rmse: 376352
[500]	valid_set's rmse: 374440
[550]	valid_set's rmse: 375214
[600]	valid_set's rmse: 373751
[650]	valid_set's rmse: 374475
[700]	valid_set's rmse: 374302
[750]	valid_set's rmse: 373696
[800]	valid_set's rmse: 373624
[850]	valid_set's rmse: 374124
[900]	valid_set's rmse: 374845
[950]	valid_set's rmse: 374618


	Ran out of time, early stopping on iteration 972. Best iteration is:
	[792]	valid_set's rmse: 373543
Saving /content/california-comp-filesAutoGluonModels/models/LightGBMXT/model.pkl
Saving /content/california-comp-filesAutoGluonModels/utils/attr/LightGBMXT/y_pred_proba_val.pkl
	-373542.6479	 = Validation score   (-root_mean_squared_error)
	89.4s	 = Training   runtime
	0.64s	 = Validation runtime
	3886.7	 = Inference  throughput (rows/s | 2500 batch size)
Saving /content/california-comp-filesAutoGluonModels/models/trainer.pkl
Skipping LightGBM due to lack of time remaining.
Saving /content/california-comp-filesAutoGluonModels/models/trainer.pkl
Skipping RandomForestMSE due to lack of time remaining.
Saving /content/california-comp-filesAutoGluonModels/models/trainer.pkl
Skipping CatBoost due to lack of time remaining.
Saving /content/california-comp-filesAutoGluonModels/models/trainer.pkl
Skipping ExtraTreesMSE due to lack of time remaining.
Saving /content/california-comp-filesAutoGlu

*** Summary of fit() ***
Estimated performance of each model:
                 model      score_val              eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0           LightGBMXT -373542.647862  root_mean_squared_error       0.643224  89.400067                0.643224          89.400067            1       True          1
1  WeightedEnsemble_L2 -373542.647862  root_mean_squared_error       0.644262  89.416964                0.001038           0.016897            2       True          2
Number of models trained: 2
Types of models trained:
{'WeightedEnsembleModel', 'LGBModel'}
Bagging used: False 
Multi-layer stack-ensembling used: False 
Feature Metadata (Processed):
(raw dtype, special dtypes):
('category', [])                    :    3 | ['Type', 'Region', 'City']
('category', ['text_as_category'])  :   15 | ['Address', 'Summary', 'Heating', 'Cooling', 'Parking', ...]
('float', [])                       :   17 | [

In [7]:
test_data = pd.read_csv(directory+'/test.csv')


In [8]:
test_data.head()

,Id,Address,Summary,Type,Year built,Heating,Cooling,Parking,Lot,Bedrooms,...,Parking features,Tax assessed value,Annual tax amount,Listed On,Listed Price,Last Sold On,Last Sold Price,City,Zip,State
0,47439,3034 N Coolidge Ave,Live within steps to the scenic views on the L...,SingleFamily,2020.0,Central,Central Air,Tandem Uncovered,940.0,2,...,Tandem Uncovered,NaN,NaN,2020-11-06,799900.0,2020-07-01,819000.0,Dodgertown,90090,CA
1,47440,565 Kenilworth Ave,duplex fixer. Input for comps only,SingleFamily,1924.0,Natural Gas,NaN,Detached,10018.8,3,...,Detached,521977.0,7494.0,2014-04-04,479950.0,2020-11-03,15000.0,San Leandro,94577,CA
2,47441,3028 N Coolidge Ave,Live within steps to the scenic views on the L...,SingleFamily,2020.0,Central,Central Air,Tandem Uncovered,940.0,2,...,Tandem Uncovered,NaN,NaN,2020-11-12,839900.0,NaN,NaN,Los Angeles,90039,CA
3,47442,3022 N North Coolidge Ave,Live within steps to the scenic views on the L...,SingleFamily,2020.0,Central,Central Air,Tandem Uncovered,940.0,2,...,Tandem Uncovered,442800.0,5370.0,2020-11-06,809900.0,2020-09-21,810000.0,Dodgertown,90090,CA
4,47443,2515 Admiral Cir,"This beautiful, spacious home built in 2017 is...",SingleFamily,2017.0,Forced Air,Central Air,"Garage Door Opener, Attached",2613.6,4,...,"Garage Door Opener, Attached",965282.0,12912.0,2020-12-02,1095000.0,2019-12-27,1041000.0,Hayward,94545,CA


In [9]:
pred_value = predictor.predict(test_data);

Loading: /content/california-comp-filesAutoGluonModels/models/LightGBMXT/model.pkl
Loading: /content/california-comp-filesAutoGluonModels/models/WeightedEnsemble_L2/model.pkl


In [10]:
pred_value.head()


,Sold Price
0,8.050001e+05
1,6.296076e+05
2,8.225619e+05
3,8.203272e+05
4,1.270101e+06


In [11]:
submission = pd.read_csv(directory+'/sample_submission.csv')
submission['label'] = pred_value
submission.head()
submission.to_csv(directory+'/my_submission.csv', index=False)